In [ ]:
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
import csv
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
ps=PorterStemmer()


import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
from transformers import AutoModel, AutoTokenizer
from collections import defaultdict
import json

"""
2 corpora - before and after. split().-> bag of words - remove their names to @user for all of them within the content of the tweet not the person who tweeted. Pull the pre-trained bert vectors - hugging face - interate through the list - get the vectors of each words, dont remove duplicates (if dont find vectors for a word - have an output for those words - see if theres any spelling). we can also lemantize them, try to get hte vectors for those lemna, if still no vector - output, keep them somewhere. 
Dictionary: {Day: {tweet_id: {word}: []}, {word []}, etc.}

Once we have the list of words of dehumanizing language, count/number of words in your corpus: (e.g.: 2% were dehumanizing before, 7% after, frequency for each day and have a time-series analysis)
"""

In [ ]:
def remove_stop_words(text: string):
    return ' '.join([item for item in text.split() if item not in stopwords.words('english')])

def remove_punctuations(text: string):
    return "".join([char for char in text if char not in string.punctuation])

def delete_usernames(text: str) -> str:
    pattern = r'@\w+'
    return re.sub(pattern, '@user', text)

# Function to split "After Corpus" into individual sentences and create a new DataFrame
def split_sentences(df, status: str):

    abbreviations = {
        "U.S.": "United States",
        "U.K.": "United Kingdom",
        "e.g.": "for example,",
        "i.e.": "such as,",
        # "Dr.": "DR_PLACEHOLDER",
        # "Mr.": "MR_PLACEHOLDER",
        # "Ms.": "MS_PLACEHOLDER",
        # "Mrs.": "MRS_PLACEHOLDER",
        # "Jr.": "JR_PLACEHOLDER",
        # "Sr.": "SR_PLACEHOLDER",
        "etc.": "..."
    }

    # Initialize an empty list to store the rows of the new DataFrame
    rows = []

    # Iterate through each row in the original DataFrame
    for _, row in df.iterrows():
        # Extract the relevant data
        subject_id = row['Subject ID']
        tweet_id = row['Tweet ID']
        corpus = row[f'{status} Corpus']

        # Replace abbreviations with placeholders
        for abbr, placeholder in abbreviations.items():
            corpus = corpus.replace(abbr, placeholder)
        
        # Split the corpus into sentences
        sentences = re.split(r'(?<=[.!?])\s+', corpus)

        # Add each sentence as a new row in the list, keeping track of the tweet ID and subject ID
        for sentence in sentences:
            rows.append({'Subject ID': subject_id, 'Tweet ID': tweet_id, 'Sentence': sentence})

    # Create a new DataFrame from the list of rows
    new_df = pd.DataFrame(rows)
    return new_df

# def stem_text(text: string):
#     return [ps.stem(word) for word in text]

In [ ]:
#Pre-process Text Data
clean_data_path = "Cleaned Data"
after = pd.read_csv(f"{clean_data_path}/After_NN_Cleaned.csv")
before = pd.read_csv(f"{clean_data_path}/Before_NN_Cleaned.csv")

# after_corpus = after[["After Corpus", "Tweet ID"]].copy()
# before_corpus = before[["Before Corpus", "Tweet ID"]].copy()

# # Apply all cleaning functions to the After Corpus column
# after_corpus["Clean Text"] = (after_corpus["After Corpus"]
#                              .apply(delete_usernames)
#                              .apply(remove_punctuations)
#                              .apply(remove_stop_words))
# before_corpus["Clean Text"] = (before_corpus["Before Corpus"]
#                              .apply(delete_usernames)
#                              .apply(remove_punctuations)
#                              .apply(remove_stop_words))

In [ ]:
# Apply the function to split sentences
after_sentences = split_sentences(after, "After")
before_sentences = split_sentences(before, "Before")
after_sentences.to_csv("Tokenizing testing.csv")

# Show the first few rows of the resulting DataFrame
after_sentences.head()

error: look-behind requires fixed-width pattern

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# after_corpus["Tokenized Text"] = 